In [0]:
from IPython import get_ipython


 # Homework #0 (Due 09/11/2019, 11:59pm)
 ## Review of Stastistical Modeling and Scientific Computing

 **AM 207: Advanced Scientific Computing**<br>
 **Instructor: Weiwei Pan**<br>
 **Fall 2019**

 **Name:** Theo Guenais

 **Students collaborators:** William Palmer (for Part 2)

 ### Instructions:

 **Submission Format:** Use this notebook as a template to complete your homework. Please intersperse text blocks (using Markdown cells) amongst `python` code and results -- format your submission for maximum readability. Your assignments will be graded for correctness as well as clarity of exposition and presentation -- a “right” answer by itself without an explanation or is presented with a difficult to follow format will receive no credit.

 **Code Check:** Before submitting, you must do a "Restart and Run All" under "Kernel" in the Jupyter or colab menu. Portions of your submission that contains syntactic or run-time errors will not be graded.

 **Libraries and packages:** Unless a problems specifically asks you to implement from scratch, you are welcomed to use any `python` library package in the standard Anaconda distribution.

In [0]:
### Import basic libraries
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')


 ## Part I: Maximum Likelihood Estimators for Linear Regression
 ### (Material covered in CS109A or equivalent)
 In this problem, you are given a dataset with a single predictor $X$ and a single outcome $y$, and your task is to fit a linear model to the data. The dataset is saved as `HW0_data.csv` in CSV (Comma Separated Values) format and can be read using the `.read_csv()` function from the `pandas` library.

 Assume that the outcome $y$ can be modeled by the following process:
 \begin{aligned}
 y &= f(x) + \epsilon = w_1x + w_0 + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 0.5)
 \end{aligned}
 where $w_1$ and $w_0$, the *parameters* of the function $f$, are unknown constants.

 1. **(Model Building)** Write the analytical expression for the likelihood function $p(y|x, w_1, w_0)$, for a single observation $(x, y)$. Write the analytical expression for the likelihood function $\prod_{n=1}^N p(y_n|x_n, w_1, w_0)$ for an entire dataset of $N$ number of observations.

   *Hint: for a given $x$ and fixed parameters, what is the distribution of $y$?*

 Let
 $y = f(x) + \epsilon = w_1x + w_0 + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 0.5)$.

 For any given $x$, $w_0$, $w_1$ in $\mathbb{R}$, $f(x)$ is a constant and so $y $ follows a normal distribution: $$\quad y \sim \mathcal{N}(f(x), 0.5)$$

 Therefore, the likelihood function is given by the product of the single-observation likelihoods (because we suppose that we have i.i.d $(\epsilon_{i})_{i} \sim \mathcal{N}(0, 0.5) )$ that is to say:
 $\begin{align}
 \prod_{n=1}^Np(y_n|x_n, w_1, w_0)
 =\frac{1}{\sqrt{0.5*2\pi}^N} \exp\left\{-\frac{\sum_{i=1}^N(y_i - (f(x_{i})))^2}{2 * 0.5} \right\}\\
 =\frac{1}{\pi^{N/2}} \exp\left\{-\sum_{i=1}^N(y_i - (w_1x_i + w_0))^2 \right\}
 \end{align}$



 2. **(Defining a Learning Objective)** The Maximum Likelihood Estimators (MLE) of $w_1$ and $w_0$ are defined as

 \begin{aligned}
 w^{\text{MLE}}_1, w^{\text{MLE}}_0 = \underset{w_1, w_0}{\mathrm{argmax}}\; \prod_{n=1}^N p(y_n|x_n, w_1, w_0)
 \end{aligned}
 Show that finding $w_1$ and $w_0$ that maximizes the likelihood is equivalent to finding parameters that minimize the Mean Squared Error (MSE) of your model.

 *Hint: show that maximizing the log likelihood is equivalent to minimizing the MSE.*

 **Answer**

 We denote by $l_{n}(\omega_{0},\omega_{1}) = log(\prod_{n=1}^N p(y_n|x_n, w_1, w_0)$ the log-likelihood of the model, defined when the product above is strictly positive.

 The logarithm function is a continuous and increasing function, so optimizing the likelihood with respect to $\omega_{0},\omega_{1} $ is equivalent to optimizing $l_{n}$.

 Furthermore, $l_{n}(\omega_{0},\omega_{1}) =-n \frac{1}{n}\sum_{i=1}^N(y_i - (w_1x_i + w_0))^2 + C$ (for some constant C), which is inversely proportional to the MSE of the model.

 Maximizing a negative function is equivalent to minimizing its absolute value.
 The likelihood function is positive, and the log-likelihood negative, so maximizing the log-likelihood is equivalent to minimizing $|l_{n}|= n.MSE +C $, hence the result.



 3. **(Model Fitting)** Analytically derive the formulae for MLE of $w_1$ and $w_0$. Describe what information you would need in order ***prove*** that the parameters you derived maximizes the likelihood (or minimizes the MSE)?

   **Extra Credit:** express your derivation of the MLE in matrix notation (i.e. [perform matrix calculus](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf)).

   *Hint: this part requires multivariate calculus.*

 **Answer**

 Let $X = \left(\begin{matrix}
  1 & .. & 1 \\
  x_{0} &...& x_{n}
  \end{matrix}
  \right)^{T}$,
  $Y=(y_{0}, .., y_{n})^{T} $ and
  $\omega =(\omega_{0}, \omega_{1})^{T}$

 We have:
 $$MSE(\omega) = \frac{1}{n}||Y-X\omega||^{2} $$ and because a norm is indefinitely differentiable, we obtain by applying the formula for the derivation of a norm (more precisely a scalar product):
 $$\nabla_{\omega}MSE = \frac{2}{n} X^{T}(Xw-Y)$$
 The critical points are such that $X^{T}X \omega^{*} = Y $ that is to say, if $X^{T} X$ is invertible: $\omega^{*} = (X^{T}X)^{-1} Y$.

 First, information about the invertibility of the matrix is required to be able to write the formula above (although we can still consider the pseudo-inverse if the matrix is not invertible).


 Second, information about the second derivatives may allow us to know whether the critical point is a minimum, a maximum (or none). In our case,  the hessian matrix is:

 $$ H = \frac{2}{n} X^{T}X $$ which is semi-definite negative due to the convexity of the norm which would mean that the point would be a minimum of our function.

 Remark: The invertibility of the matrix is actually linked to the rank of X.


 4. **(Model Evaluation)** Using the `LinearRegression` class from `sklearn.linear_model`, fit a linear regression model to the dataset in `HW0_data.csv`. When you call the `.fit()` function of `LinearRegression`, the default is to find parameters that minimize MSE.

   Visualize the linear function you fitted as well as the data in one plot.
   Visualize a histogram of the residuals.
   Using both plots, argue for whether or not your model is appropriate for the data.

   *Hint: look at the assumptions you've made in your modeling process, which assumptions are supported by the data, which are contradicted by the data?*

In [0]:
import sklearn
colab = False
#%cd HW0
if colab:
  from google.colab import files
  uploaded_files = files.upload()
else:
    pass

df = pd.read_csv("HW0_data.csv")



In [0]:
#Regression
x = df['x'].to_numpy().reshape(-1,1) ; y = df['y'].to_numpy().reshape(-1,1)
linear_regression = sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False)
linear_regression.fit(x,y)
infer = linear_regression.predict(x)
#Visualization
fig, ax = plt.subplots(1,3, figsize=(20,5))
#Not required. Uncomment
#ax[0].plot(y, label='y')
#ax[0].plot(infer, label='predicted y')
#ax[0].legend()
#ax[0].set_title("Superposition of real target and predicted target values")

#Scatter plot
ax[0].scatter(x,y, label='Observations')
ax[0].plot(x,infer,color='red', label='Linear model prediction')
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[0].legend()
ax[0].set_title("True target and predicted target values")
#histogram of the residuals
ax[1].hist((infer-y)**2, bins=20)
ax[1].set_xlabel('Residual values')
ax[1].set_ylabel('Count')
ax[1].set_title("Histogram of the residuals")
#histogram of the error
ax[2].hist((infer-y), bins=15)
ax[2].set_xlabel('Error values')
ax[2].set_ylabel('Count')
ax[2].set_title("Histogram of the error")

print('Mean and variance of the error: {}, {}'.format(
    np.mean(infer-y), np.var(infer-y)))
print("Estimated regression coefficients: intercept {}, coefficient {}".format(
    linear_regression.intercept_, linear_regression.coef_[0]))


 The modeling assumption about the linearity of the model doe not seem unplausible. However, the error looks skewed on left side and it is not obvious to say that error looks distributed like a Gaussian distribution (or the residuals like a Chi-Squared distribution).

 Indeed, we can observe different points at the extremity of the observation space (x,y-coordinates) that might bias the fit of the model and let us think that the error is not uncorrelated to the variable x. Furthermore, the variance of the error seems not to match the 0.5 variance assumption.

 5. **(Uncertainty Quantification)** Compute the 95% [Confidence Interval (CI)](https://stattrek.com/estimation/confidence-interval.aspx) of the parameters you estimated for the dataset in `HW0_data.csv` by bootstrapping (this requires you to access the model parameters of the `LinearRegression` class). How confident are you in the 'correctness' or 'reliability' of your parameter estimates given the CI's?

   Compute the [Predictive Interval (PI)](https://otexts.com/fpp2/prediction-intervals.html) of a single prediction at $x=2$ by bootstrapping. How confident are you in your prediction for $x=2$ given the PI?

   *Hint: bootstrapping is the process of (1) generating multiple datasets (with equal size as the training data) that are similarly distributed as your training data by sampling your data with replacement, (2) estimating a quantity of interest repeatedly using these datasets. You can then empirically compute the percentile of interest from the bootstrap estimates of your quanity of interest.*

   *Generating samples from your training data can be done using the `.sample()` function of the `pandas` `DataFrame` class.*

   *Computing percentiles of samples can be done using the `.percentile()` function of the `numpy` library.*

In [0]:
#Boootstrap    
verbose = False
N_bootstrap = 2000
intercepts = []
coeffs = []
y_pred = []

#store boostrap values in the two lists above
for i in range(N_bootstrap):
    bootstrap = df.sample(frac=0.9, replace=True)
    x = bootstrap['x'].to_numpy().reshape(-1,1)
    y = bootstrap['y'].to_numpy().reshape(-1,1)
    linear_regression.fit(x,y)
    intercepts.append(linear_regression.intercept_)
    coeffs.append(linear_regression.coef_[0])
    y_pred.append(linear_regression.predict( np.array([2]).reshape(-1,1)))

#Compute the confidence interval 
alpha = 2.5 # 1/2 percentage we want
w1_start = np.percentile(coeffs, alpha); w0_start = np.percentile(intercepts, alpha)
w1_end = np.percentile(coeffs, 100-alpha); w0_end = np.percentile(intercepts, 100-alpha)
print("{}% confidence interval for w0: [{},{}]" .format((100-2*alpha), w0_start,w0_end))
print("{}% confidence interval for w1: [{},{}]" .format(100-2*alpha, w1_start,w1_end))

#Compute the prediction 
pi_start = np.percentile(y_pred, alpha)
pi_end = np.percentile(y_pred, 100-alpha)
print("{}% prediction interval: [{},{}]" .format(100-2*alpha, pi_start,pi_end))
    


 The notions of "correctness" or "reliability" are quite complex and in this application, they are pretty subjective. This being said, the confidence interval (as well as the prediction interval) seems large, and the estimation seems too variable to be able to assert anything about the predictions with certitude. The several elements that we discussed in the previous question (as well as the small sample size) are likely to be the reason for this uncertainty and the fact that at the end, our model does not seem "reliable".

 ## Part II: Bayesian Linear Regression
 ### (Material covered in CS109 B or equivalent)
 In this problem, your task is to perform Bayesian linear regression on the dataset in `HW0_data.csv`.

 Assume that the outcome $y$ can be modeled by the following process:
 \begin{aligned}
 y &= f(x) + \epsilon = w_1x + w_0 + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 0.5)\\
 w_0 &\sim \mathcal{N}(0, 1)\\
 w_1 &\sim \mathcal{N}(0, 0.5)
 \end{aligned}
 In this model, we assume the same likelihood as in Part I, but we also include priors for $w_0$ and $w_1$: $p(w_0) = \mathcal{N}(0, 1)$, $p(w_1) = \mathcal{N}(0, 0.5)$

 1. **(Inference)** Analytically derive the joint posterior distribution over the parameters. That is, analytically compute

 \begin{aligned}
 p(w_1, w_0 | y_1, \ldots y_n, x_1, \ldots, x_n) = \frac{\left(\prod_{n=1}^Np(y_n|x_n, w_1, w_0)\right)p(w_1)p(w_0)}{\prod_{n=1}^Np(y_n|x_n)}.
 \end{aligned}


   *Hint: The posterior of a model with a Gaussian (or normal) likelihood and Gaussian priors is yet again Gaussian. This derivation is included in many machine learning textbooks like Machine Learning: a Probabilistic Perspective by Murphy or Pattern Recognition and Machine Learning by Bishop. This derivation also appears online in many course notes - you are welcomed to use any and all available resources.*

 file.jpeg![image.png](attachment:image.png)

 **1.1-Answer**

 With $C=\left(\begin{array}{cc}
 1/2 & 0 \\
 0 & 1 \\
 \end{array}\right)\,$
 and
 $ \Sigma^{-1} = 2 (\mathbf{X}^T\mathbf{X} + C)$


 We obtain
 $$\mu = \Sigma\mathbf{X}^T\mathbf{Y}
 =\left(\mathbf{X}^T\mathbf{X} + C\right)^{-1} \mathbf{X}^{T}\mathbf{Y}$$

 and
 $$\Sigma = \frac{1}{2} (X^{T}X + C)^{-1} $$
 $$\Sigma = \frac{1/2}{(n+\frac{1}{2})[(\sum_{n=1}^Nx_n^2)+1]-(\sum_{n=1}^N x_n)^2}
 \left(\begin{matrix}
 (\sum_{n=1}^Nx_n^2)+1 & \sum_{n=1}^N x_n \\
 \sum_{n=1}^N x_n & n +\frac{1}{2}
 \end{matrix}
 \right)
 $$

 NEw command just to check
 $$ f(x) = \Sigma + \mu

 2. **(Model Interpretation)** Visualize the ***joint posterior*** over the parameters, $p(w_1, w_0 | y, x)$ for the dataset in `HW0_data.csv` (you may use a scatter plot of samples drawn from the posterior or a contour map/heat map of the pdf). In separate plots, visualize the ***marginal posterior*** distribution over each parameter,


 \begin{aligned}
 p(w_1 | y_1, \ldots y_n, x_1, \ldots, x_n) &= \int_{w_0} p(w_1, w_0 | y_1, \ldots y_n, x_1, \ldots, x_n) dw_0\\
 p(w_0 | y_1, \ldots y_n, x_1, \ldots, x_n) &= \int_{w_0} p(w_1, w_0 | y_1, \ldots y_n, x_1, \ldots, x_n) dw_1
 \end{aligned}

   Is the information contained in the joint posterior $p(w_1, w_0 | y_1, \ldots y_n, x_1, \ldots, x_n)$ equivalent to the information contained in the two marginals $p(w_1 | y_1, \ldots y_n, x_1, \ldots, x_n), p(w_0 | y_1, \ldots y_n, x_1, \ldots, x_n)$? That is, what does each distribution tell you?

   *Hint: it is not necessary to analytically derive the marginals $p(w_i | y_1, \ldots y_n, x_1, \ldots, x_n)$. Recall that you can perform marginalization of a variable by sampling from the joint distribution and disregarding that variable.*

In [0]:
import scipy as sp
#Define X with the intercept, Y and compute the expectations and 
X_0 = df['x'].to_numpy().reshape(-1,1)
Y = df['y'].to_numpy()
X = np.concatenate((np.ones(X_0.shape), X_0), axis=1).reshape(-1,2)
C = np.array([[0.5,0],[0,1]])

inv_cov = 2 * (np.dot(X.T,X) + C) 
sigma = np.linalg.inv(inv_cov) #Covariance matrix
mu = 2 * np.dot(np.dot(sigma, X.T) ,Y) #Expectation
print("Expectation vector: {}".format(mu))
print("Covariance matrix: {}".format(sigma))


x, y = np.mgrid[8.5:9.1:.01, 5.12:5.325:.001]
pos = np.dstack((x, y))
z = sp.stats.multivariate_normal(mu, sigma).pdf(pos)

#mu = mu.reshape(1,-1)[0]
#define a grid and then sample
n_sample = 10000 #Number of samples
samples_omegas = np.random.multivariate_normal(mu, sigma, size=n_sample)  

#Visualize
fig, ax = plt.subplots(1, 2, figsize=(20, 5))
#plot the gaussian pdf as a countour map
ax[0].contour(x, y, z, levels=14, linewidths=0.5, colors='red')
ax[0].contourf(x, y, z, levels=14, cmap='Reds')
ax[0].set_title('Contour plot of our posterior gaussian distribution')
ax[0].set_xlabel('size')
ax[0].set_ylabel('weight')
#Scatter plotting of the samples
ax[1].scatter(samples_omegas[:, 0], samples_omegas[:, 1], color='red', alpha=0.1)
ax[1].set_title('Scatter plot of the sampled joint posterior distribution')
ax[1].set_xlabel('w_0')
ax[1].set_ylabel('w_1')

fig, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].hist(samples_omegas[:, 0], bins=100)
ax[0].set_xlabel('Values')
ax[0].set_ylabel('Count')
ax[0].set_title("Histogram of the (marginalized) posterior intercept")

ax[1].hist(samples_omegas[:, 1], bins=100)
ax[1].set_xlabel('Values')
ax[1].set_ylabel('Count')
ax[1].set_title("Histogram of the (marginalized) posterior coefficient")
plt.show()


 $\omega | x,y = \left(\begin{array}{c}
 w_{0} \\
 w_{1}
 \end{array} |x,y\right)$ is a Gaussian vector of parameters $\mu, \Sigma$.


 Therefore, $w_{0}|x,y =\left(\begin{array}{cc}
 1 & 0
 \end{array}\right) \omega$ is also a Gaussian vector of parameters:

 $\left(\begin{array}{cc}
 1 & 0
 \end{array}\right)\mu $  and
 $\left(\begin{array}{cc}
 0 & 1
 \end{array}\right)\Sigma
 \left(\begin{array}{c}
 1\\
 0
 \end{array}\right)$ that we denote $\mu_{0}$ and $\Sigma_{11}$.

 The same argument holds for
 $w_{1}|x,y =\left(\begin{array}{cc}
 0 & 1
 \end{array}\right) \omega$, normally distributed with parameters
 $\mu_{1}$ and $\Sigma_{22}$.


 Therefore, the marginal distributions allow us to reconstruct the expectation of the joint distribution, but not the covariance matrix, due to the loss of information about the correlation coefficient $\Sigma_{12} = \Sigma_{21} $(during the integration process). Experimentally, this is a loss of information in terms of the inclination of the contour lines or the spread of the scatter plot.

 Remark: However, if the two random variables were independent, id est $\Sigma_{12} = 0$ (a covariance equal to zero is equivalent to independence in the case of gaussian vectors), we would not lose any infommation with the integration.

 We design the following experiment to have an experimental proof of what was previously said:

 We marginalize the two variables of interest, (shuffle the associated vectors) and show a scatter plot of those points, which are actually supposed to be drawn from the bivariate distribution $p(\omega_{0}|x,y) \otimes p(\omega_{1}|x,y)$.






In [0]:
#Shuffle the estimated coefficients independently 
omega0 = np.random.permutation(samples_omegas[:,0]).reshape(-1,1) 
omega1 = np.random.permutation(samples_omegas[:, 1]).reshape(-1,1)

shuffled_omegas = np.concatenate((omega0, omega1), axis=1)

plt.scatter(omega0, omega1, color='blue')
plt.xlabel('w_0')
plt.ylabel('w_1')
plt.title("Scatter plot of the shuffled coefficients")
plt.show()


 3. **(Prediction)** In Bayesian linear regression, we obtain a distribution over possible parameters given the data, $p(w_1, w_0 | y_1, \ldots y_n, x_1, \ldots, x_n)$, rather than a single ***point-estimate*** of the 'best' parameters. This means that when we predict an outcome for, say, $x=2$ we get a distribution over possible values of $y$ rather than a single $y$. The distribution over the predicted $y$ for a given $x$ is called the ***posterior predictive***, and is denoted $p(y|x)$.

   Visualize the posterior predictive $p(y|x)$ at $x=2$ for the dataset in `HW0_data.csv`as a histogram.

   *Hint: randomly sample possible parameters from the posterior $p(w_1, w_0 | y_1, \ldots y_n, x_1, \ldots, x_n)$. Each sample represents a linear function $f(x)$. Make a prediction at $x=2$ using each $f(x)$.*

In [0]:
X = np.array([1,2]).reshape(-1,1) #include intercept
Y_pred = np.dot(samples_omegas, X) #use the sample omegas previously computed
plt.hist(Y_pred,bins=100)
plt.title("Histogram of the samples from the posterior predictive distribution at x=2")
plt.xlabel('Value')
plt.ylabel('Count')
plt.show()


 ## Part III: Comparison

 You've now modeled the same set of data using non-Bayesian and Bayesian regression models. In both paradigms it's possible to identify a 'best' set of model parameters (in the non-Bayesian approach, we compute the MLE; in the Bayesian approach, we can compute the mean or mode of the posterior). In both paradigms it is possible to get a sense of the uncertainty we have about our estimates of the parameter and our predictions.

 1. Compute the 95 percentile interval of the marginal posterior distributions $p(w_i | y_1, \ldots y_n, x_1, \ldots, x_n)$  for the dataset in `HW0_data.csv` (you can either do this analytically, or empirically using samples from $p(w_i | y_1, \ldots y_n, x_1, \ldots, x_n)$). Compare these intervals to the CI's you computed in Part I. Which interval is wider? What does this say about the uncertainty of each model over the parameters?

In [0]:
alpha = 2.5
#omega0 and omega1 are the coefficients from the quesstion above
w1_start = np.percentile(omega1, alpha); w0_start = np.percentile(omega0, alpha)
w1_end = np.percentile(omega1, 100-alpha); w0_end = np.percentile(omega0, 100-alpha)
print("95 percentile interval for the marginal posterior "      "distribution of omega 0: [{} , {}]".format(w0_start, w0_end))
print("95 percentile interval for the marginal posterior "      "distribution omega 1: [{} , {}]".format(w1_start, w1_end))


 The Bayesian interval is much narrower than the one previously computed. The Bayesian posterior distribution actually represents a "balance" between the sampling distribution and the prior distribution(s). When the number of observations increases, so does the importance of the sampling distribution and reversely. In this application, the dataset can be considered as relatively small and the bayesian posterior intervals are dominated by the importance of the prior distributions. Furthermore, those prior distributions have localized masses; their variances are small indeed and therefore their 95% confidence interval are narrow as well. This explains the reasons why the bayesian interval is much narrower than the "MLE interval". With less strong assunptions about the prior distributions, it could have been very different.


 2. Compute the 95 percentile interval of posterior predictive at $x=2$ for the dataset in `HW0_data.csv` (you can either do this analytically, or empirically using samples from $p(y| x=2)$). Compare this interval to the PI you computed in Part I. Which interval is wider? What does this say about the uncertainty of each model over the prediction?

In [0]:
#From Part II-Q3 we have
#X = np.array([1,2]).reshape(-1,1) 
#Y_pred = np.dot(samples_omegas, X)
alpha=2.5
pi_start = np.percentile(Y_pred, alpha); pi_end = np.percentile(Y_pred, 100-alpha)
print('95% predicton interval: [{},{}]'.format(pi_start, pi_end))


 Simililarly, the prediction interval for the Bayesian model is much narrower. Given the data and the prior distribution, we seem much more confident in out prediction. However, one should remember the strength of the modelling assumptions made (Gaussian priors with a small variances). The small variances indicate that the modelling step is already confident in these assumptions, otherwise this would simply "bias" the model and give too much importance to "false" prior beliefs.
 Nevertheless, the first model was also subject to criticism (cf part I) for several reasons and due to the spread of the intervals, we were not really confident in out estimation.

 **Extra Credit:** For the maximum likelihood model, visualize the mean prediction and the PI at 100 equally spaced $x$'s (using bootstrap) against the training data. Scatter plot the data; plot the mean prediction as a line; plot the PI as similarly colored shaded region (use the `.fill_between()` function from the `matplotlib`).

 For the Bayesian regression model, visualize the posterior mean prediction and the 95 percentile interval of the posterior predictive (you should estimate these quantities empirically). Scatter plot the data; plot the posterior mean prediction as a line; plot the PI as similarly colored shaded region.

 Using these visualizations, make an argument for which model is more appropriate for the model.

 For each model how do your choices (the variance in the likelihood, the priors) affect the model you learn? Given the visualizations, are our choices of the likelihood (and or the priors) appropriate?

In [0]:
#Maximum likelihood model
xs = np.linspace(-12, 12, 100)

def predict_bootstrap(df, xs, N_bootstrap=20):
    #Returns a list of size N_bootsrap with elements being arrays of size xs.shape
    intercepts = []
    coeffs = []
    y_pred = []
    for i in range(N_bootstrap):
        bootstrap = df.sample(frac=0.9, replace=True)  
        x = bootstrap['x'].to_numpy().reshape(-1,1)
        y = bootstrap['y'].to_numpy().reshape(-1,1)
        #linear_regression = sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False)
        linear_regression.fit(x,y)
        #intercepts.append(linear_regression.intercept_)
        #coeffs.append(linear_regression.coef_[0])
        y_pred.append(linear_regression.predict(xs.reshape(-1,1)))
    assert len(y_pred) == N_bootstrap
    assert y_pred[0].shape[0] == xs.shape[0]
    return y_pred

#MLE model
Y_pred_bs = predict_bootstrap(df, xs, N_bootstrap=20) #Bootsrapped prediction
mean_pred_bs = np.mean(Y_pred_bs, axis=0) #mean prediction
#confidence intervals
alpha = 2.5
ci_start = np.percentile(Y_pred_bs, alpha, axis=0)
ci_end = np.percentile(Y_pred_bs, 100-alpha, axis=0)

#Bayes model
#samples_omegas #we have it
xs = xs.reshape(-1,1)
X_bayes = np.concatenate([ np.ones((xs.shape[0],1)), xs], axis=1)
Y_pred_bayes = np.dot(samples_omegas, X_bayes.T)

mean_pred_bayes = np.mean(Y_pred_bayes, axis=0)
ci_start_bayes = np.percentile(Y_pred_bayes, alpha, axis=0)
ci_end_bayes = np.percentile(Y_pred_bayes, 100-alpha, axis=0)



In [0]:
fig, ax =  plt.subplots(1, 2, figsize=(20, 5))
ax[0].plot(xs[:,0], mean_pred_bs, color='black', label='Mean prediction for the bootsrap MLE model')
ax[0].scatter(df['x'], df['y'], label='Observations')
ax[0].fill_between(xs[:,0], ci_start[:,0], ci_end[:,0], color='orange', label='95 %PI')
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[0].legend()
ax[0].set_title("Bootsrapped Mean prediction and 95% PI for the MLE model")

ax[1].plot(xs[:,0], mean_pred_bayes, color='black', label='Mean prediction for the bootsrap MLE model')
ax[1].scatter(df['x'], df['y'], label='Observations')
ax[1].fill_between(xs[:,0], ci_start_bayes, ci_end_bayes, color='orange',label='95 %PI')
ax[1].set_xlabel('x')
ax[1].set_ylabel('y')
ax[1].legend()
ax[1].set_title("Mean prediction and 95% PI for the Bayesian model")

plt.show()


 Using these visualizations, make an argument for which model is more appropriate for the model.

 For each model how do your choices (the variance in the likelihood, the priors) affect the model you learn? Given the visualizations, are our choices of the likelihood (and or the priors) appropriate?

 The previous answers partially covered the fact that small variances of the prior distributions strongly influence the Bayesian model and dominate the importance of the data. The Bayesian priors are only slighlty updated by the data and the posterior distribution seems way too "confident" even though the graphs visually show us that the prediction intervals are too small and therefore seem inadequate.
 Even if one can criticize the MLE model (cf Part I) and especially the simplicity of the modelling assumptions, the uncertainty in the prediction seems adequate.